In [3]:
import numpy as np
import math
import random
import visualize

ModuleNotFoundError: No module named 'taichi'

In [2]:
class Camera:
    def __init__(self, fov=60, aspect_ratio=1.0):
        # Camera parameters
        self.lookfrom = np.array((0.4, 0.5, 2.0))
        self.lookat = np.array([0.0, 0.0, 0.0])
        self.vup = np.array([0.0, 1.0, 0.0])
        self.fov = fov
        self.aspect_ratio = aspect_ratio
        
        theta = self.fov * (PI / 180.0)
        half_height = math.tan(theta / 2.0)
        half_width = self.aspect_ratio * half_height
        
        self.cam_origin = self.lookfrom
        
        w = self.lookfrom - self.lookat
        w = w / np.linalg.norm(w)

        u = np.cross(self.vup, w)
        u = u / np.linalg.norm(u)
        
        v = np.cross(w, u)
        
        self.cam_lower_left_corner = self.cam_origin - half_width * u - half_height * v - w
        self.cam_horizontal = 2 * half_width * u
        self.cam_vertical = 2 * half_height * v
    
    def test(self):
        print("test")
    
    def get_direction(self, u, v):
        
        u = (u) / 800
        v = (v) / 800

        direction = self.cam_lower_left_corner + u * self.cam_horizontal + v * self.cam_vertical - self.cam_origin
        #direction = direction / np.linalg.norm(direction)
        
        print("self.cam_lower_left_corner[None]", self.cam_lower_left_corner)
        print("u", u)
        print("self.cam_horizontal[None]", u * self.cam_horizontal)
        print("self.cam_vertical[None]", v * self.cam_vertical)
        print("self.cam_origin[None]", self.cam_origin)
        
        return direction
    
    def get_impossible_intersection(self,index,u,v):
        direction = self.get_direction(u,v)
        return self.lookfrom + index*direction

In [22]:
myCamera = Camera()

foreground = myCamera.get_impossible_intersection(3,400, 500)

self.cam_lower_left_corner[None] [-0.32965573 -0.29884196  1.29564164]
u 0.5
self.cam_horizontal[None] [ 0.56613852  0.         -0.1132277 ]
self.cam_vertical[None] [-0.03369872  0.7009334  -0.16849361]
self.cam_origin[None] [0.4 0.5 2. ]


In [23]:
background = a.get_impossible_intersection(4,400, 500)

self.cam_lower_left_corner[None] [-0.32965573 -0.29884196  1.29564164]
u 0.5
self.cam_horizontal[None] [ 0.56613852  0.         -0.1132277 ]
self.cam_vertical[None] [-0.03369872  0.7009334  -0.16849361]
self.cam_origin[None] [0.4 0.5 2. ]


In [39]:
class structure:
    def __init__(self, seed, seed_next_possible):
        self.data = seed
        self.next_possibles = seed_next_possible
        print("initialized data: \n", self.data)

  
    def cost_func(self, pos):
        return random.random()
  
    def add_vertex(self, next_vertex):
        self.data = np.vstack([self.data, next_vertex])

    def remove_possible(self, next_index):
        self.next_possibles = np.delete(self.next_possibles, next_index, axis=0)
  
  #define direction:
  #x=-1 -> 0, x=1 -> 1, y=-1 -> 2, y=1 -> 3, z=-1 -> 4, z=1 -> 5,
    def get_next_possible(self, available_contact, direction):
        result = np.array([0,0,0,0])

        available_contact = np.append(available_contact, 0)
        if direction == 0 or direction == 1:
            result = np.vstack([result, available_contact + np.array([0,1,0,3])])
            result = np.vstack([result, available_contact + np.array([0,-1,0,2])])
            result = np.delete(result, 0, axis =0)
            return result

        if direction == 2 or direction == 3:
            result = np.vstack([result, available_contact + np.array([1,0,0,1])])
            result = np.vstack([result, available_contact + np.array([-1,0,0,1])])
            result = np.delete(result, 0, axis =0)
            return result

    
    def get_last_direction(self,direction):
        count =direction[1]*1 + direction[2]*2
        return count
  
    def get_vertex_forward(self, new_vertex_clean, direction):
        if direction == 0:
            return new_vertex_clean + np.array([-1,0,0])

        if direction == 1:
            return new_vertex_clean + np.array([1,0,0])

        if direction == 2:
            return new_vertex_clean + np.array([0,-1,0])

        if direction == 3:
            return new_vertex_clean + np.array([0,1,0])
    
    def generate(self, steps):
        for i in range(steps):
            print("step ", i+1)
            self.process()


    def process(self):
        #step 1: for all available contacts, assume they will make a turn  
        #print("next_possibles", self.next_possibles)

        #step2: create a 1d array that stores cost for each possible point
        costs = np.array([])
        for next_possible in self.next_possibles:
            costs = np.append(costs, self.cost_func(next_possible))

        #print("costs", costs)

        #step3 find the next_vertex with smallest cost
        next_index = np.argpartition(costs, -1)[-1:]
        next_vertex = self.next_possibles[next_index] 

        #print("next_vertex", next_vertex)

        #step4 add next_vertex to data
        new_direction = next_vertex[0][3]
        #print("new_direction", new_direction)
        new_vertex_clean = np.array([next_vertex[0][0], next_vertex[0][1], next_vertex[0][2]])
        #print("new_vertex_clean", new_vertex_clean)
        self.add_vertex(new_vertex_clean)
        
        span = 2 + int(random.random() * 5 % 5)
        for i in range(span):
            new_vertex_clean = self.get_vertex_forward(new_vertex_clean, new_direction)
            self.add_vertex(new_vertex_clean)

        print("data: \n", self.data)

        #step5 update next_possible
        self.remove_possible(next_index)
        #print("next_possibles, removed old", self.next_possibles)
        #print("next_vertex", new_vertex_clean)
        #print("new_direction", new_direction)
        new_next_possibles = self.get_next_possible(new_vertex_clean, new_direction)
        self.next_possibles = np.append(self.next_possibles,new_next_possibles, axis=0)
        #print("next_possibles, add new", self.next_possibles)


In [45]:
seed = np.array([[0,0,0], [0,1,0],[0,2,0],[0,3,0], [0,4,0]])
seed_next_possible = np.array([[-1,4,0,0],[1,4,0,1]])

myStructure = structure(seed, seed_next_possible)

myStructure.generate(5)

initialized data: 
 [[0 0 0]
 [0 1 0]
 [0 2 0]
 [0 3 0]
 [0 4 0]]
step  1
data: 
 [[0 0 0]
 [0 1 0]
 [0 2 0]
 [0 3 0]
 [0 4 0]
 [1 4 0]
 [2 4 0]
 [3 4 0]
 [4 4 0]
 [5 4 0]]
step  2
data: 
 [[ 0  0  0]
 [ 0  1  0]
 [ 0  2  0]
 [ 0  3  0]
 [ 0  4  0]
 [ 1  4  0]
 [ 2  4  0]
 [ 3  4  0]
 [ 4  4  0]
 [ 5  4  0]
 [-1  4  0]
 [-2  4  0]
 [-3  4  0]
 [-4  4  0]
 [-5  4  0]
 [-6  4  0]
 [-7  4  0]]
step  3
data: 
 [[ 0  0  0]
 [ 0  1  0]
 [ 0  2  0]
 [ 0  3  0]
 [ 0  4  0]
 [ 1  4  0]
 [ 2  4  0]
 [ 3  4  0]
 [ 4  4  0]
 [ 5  4  0]
 [-1  4  0]
 [-2  4  0]
 [-3  4  0]
 [-4  4  0]
 [-5  4  0]
 [-6  4  0]
 [-7  4  0]
 [-7  5  0]
 [-7  6  0]
 [-7  7  0]
 [-7  8  0]
 [-7  9  0]
 [-7 10  0]
 [-7 11  0]]
step  4
data: 
 [[ 0  0  0]
 [ 0  1  0]
 [ 0  2  0]
 [ 0  3  0]
 [ 0  4  0]
 [ 1  4  0]
 [ 2  4  0]
 [ 3  4  0]
 [ 4  4  0]
 [ 5  4  0]
 [-1  4  0]
 [-2  4  0]
 [-3  4  0]
 [-4  4  0]
 [-5  4  0]
 [-6  4  0]
 [-7  4  0]
 [-7  5  0]
 [-7  6  0]
 [-7  7  0]
 [-7  8  0]
 [-7  9  0]
 [-7 10  0]
 [-7 11  

In [46]:
gen_structure = myStructure.data